In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
import io
from PIL import Image
import hashlib

import pandas as pd
from tqdm import tqdm
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
'''
Code credit: 
https://medium.com/swlh/web-scraping-stock-images-using-google-selenium-and-python-8b825ba649b9
'''
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content
        
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path, hashlib.sha1(url.encode('utf-8')).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [3]:
class ContextItem:
    
    def __init__(self, ctx_item):
        self.contents = ctx_item
        self.index = None
        self.brand = None
        self.name = None
        self.price = None
        self.likes = None
        self.product_url = None
        self.image_url = None
        self.image_path = None
        
    def get_data(self, i):
        self.index = i
        
        try:
            self.brand = self.contents.find(class_='brand').get_text()
        except:
            pass
        
        try:
            self.name = self.contents.find(class_='description').get_text()
        except:
            pass
               
        try:
            self.price = float(self.contents.find(class_='price').get_text()[1:])
        except:
            pass
        
        try:
            self.likes = int(self.contents.find(class_='like-count').get_text().strip())
        except:
            pass
        
        self.product_url = 'https://shoplook.io'+ self.contents.a.get('href')
        self.image_url = self.contents.img['src']
        self.image_path = os.path.join(hashlib.sha1(self.image_url.encode('utf-8')).hexdigest()[:10] + '.jpg')
                

In [4]:
url = 'https://shoplook.io/discover/all/search?sort=trending&q=' + 'date night'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [5]:
#need to manually scroll and then form the soup
soup = BeautifulSoup(driver.page_source, 'lxml')

items = soup.find_all('a', class_='feed-outfit')
hrefs = list()
for item in items:
    hrefs.append(item.get('href'))

In [6]:
len(hrefs)

300

In [9]:
from selenium.webdriver.chrome.options import Options
import re

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

data = []
for href in tqdm(hrefs):
    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")

    set_url = 'https://shoplook.io' + href
    driver = webdriver.Chrome(chromedriver, options=options)
    driver.get(set_url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    try:
        name = soup.find('div', class_='info-header').get_text().strip()
    except:
        name = None

    image_url = soup.img['src']
    set_id = re.findall('[0-9]+', set_url)[0]

    try:
        likes = soup.find('div', class_='interactions-number').get_text()
    except:
        likes = None

    try:
        desc = soup.find('div', class_='tags').get_text().strip()

    except:
        desc = None

    items = []
    cxt_items = soup.find_all('app-product-container')
    for i, cxt_item in enumerate(cxt_items, 1):
        item = ContextItem(cxt_item)
        item.get_data(i)
        cxt_data = {'index': item.index, 'brand': item.brand, 'name': item.name, 'price': item.price, 
                'likes': item.likes, 'product_url': item.product_url, 'image_url': item.image_url,
                'image_path': item.image_path, 'set_id': set_id}
        items.append(cxt_data)
        persist_image(f'./shoplook/data/images/date_night', item.image_url)

    set_data = {'name': name, 'likes': likes, 'items': items, 'image_url': image_url,
            'set_url': set_url, 'set_id': set_id, 'desc': desc, 'category': 'date_night'}

    data.append(set_data)
    driver.quit()
    time.sleep(0.05)

with open(f'./shoplook/data/date_night_data.json', 'w') as fout:
    json.dump(data, fout)

  0%|          | 0/300 [00:00<?, ?it/s]

SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/c0d9ecb5-bbd2-4480-9cd4-2358f2e5ff4c.jpeg - as ./shoplook/data/images/date_night/814c08ef4f.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/fd93a771-3591-48f5-9557-38810e0694b0.jpeg - as ./shoplook/data/images/date_night/321731602d.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/6da27dd7-646d-404c-931f-e966a4ba8159.jpeg - as ./shoplook/data/images/date_night/bd814fbbba.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/79c7b727-39a0-40b4-8ec5-299e42c76aa0.jpeg - as ./shoplook/data/images/date_night/a76312684f.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/187731a8-c250-4fc7-969a-5feaa8373136.jpeg - as ./shoplook/data/images/date_night/67ba6a0045.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_produc

  0%|          | 1/300 [00:14<1:13:43, 14.79s/it]

SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/5e342609-60e5-4619-8fce-e8f65b6797ec.jpg - as ./shoplook/data/images/date_night/a343070d5e.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/2df621f0-2968-4fd1-8410-8204110cbab6.jpg - as ./shoplook/data/images/date_night/17ebec6622.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/99954f0a-8abf-43fe-b219-01feb6e425a2.jpeg - as ./shoplook/data/images/date_night/1924c09900.jpg


/Users/emilyng/anaconda3/envs/metis/lib/python3.8/site-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/e3505356-057c-4773-a2b2-9eeeb85fa596.jpeg - as ./shoplook/data/images/date_night/02e5fdef3a.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/206c0a11-859b-4ee1-8fbc-27c7fbf75c17.jpeg - as ./shoplook/data/images/date_night/149574772d.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/4bc975d5-d491-4001-9ae1-0c4122817938.png - as ./shoplook/data/images/date_night/0f7aa7f208.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/4e60885f-5cf0-4b4e-93bc-43d789b509c0.jpeg - as ./shoplook/data/images/date_night/b7d8c5adff.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/3cb69812-8f1d-4632-afe3-ee375c17055b.jpeg - as ./shoplook/data/images/date_night/d574855e27.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs

 27%|██▋       | 81/300 [19:01<57:18, 15.70s/it]

SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/13becfd2-0f58-4767-9204-6b1e25162077.jpeg - as ./shoplook/data/images/date_night/a95d36fff6.jpg


 27%|██▋       | 81/300 [19:04<51:34, 14.13s/it]


KeyboardInterrupt: 

In [10]:
with open(f'./shoplook/data/date_night_data.json', 'w') as fout:
    json.dump(data, fout)
    
driver.quit()

In [24]:
url = 'https://shoplook.io/discover/all/search?sort=trending&q=' + 'casual'
driver = webdriver.Chrome(chromedriver)
driver.get(url)
time.sleep(1)

In [25]:
#scroll before running this cell!
soup = BeautifulSoup(driver.page_source, 'lxml')
items = soup.find_all('a', class_='feed-outfit')
hrefs = list()
for item in items:
    hrefs.append(item.get('href'))

In [26]:
len(hrefs)

250

In [28]:
from selenium.webdriver.chrome.options import Options
import re

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

data = []
for href in tqdm(hrefs):
    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")

    set_url = 'https://shoplook.io' + href
    driver = webdriver.Chrome(chromedriver, options=options)
    driver.get(set_url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    try:
        name = soup.find('div', class_='info-header').get_text().strip()
    except:
        name = None

    image_url = soup.img['src']
    set_id = re.findall('[0-9]+', set_url)[0]

    try:
        likes = soup.find('div', class_='interactions-number').get_text()
    except:
        likes = None

    try:
        desc = soup.find('div', class_='tags').get_text().strip()

    except:
        desc = None

    items = []
    cxt_items = soup.find_all('app-product-container')
    for i, cxt_item in enumerate(cxt_items, 1):
        item = ContextItem(cxt_item)
        item.get_data(i)
        cxt_data = {'index': item.index, 'brand': item.brand, 'name': item.name, 'price': item.price, 
                'likes': item.likes, 'product_url': item.product_url, 'image_url': item.image_url,
                'image_path': item.image_path, 'set_id': set_id}
        items.append(cxt_data)
        persist_image(f'./shoplook/data/images/casual', item.image_url)

    set_data = {'name': name, 'likes': likes, 'items': items, 'image_url': image_url,
            'set_url': set_url, 'set_id': set_id, 'desc': desc, 'category': 'casual'}

    data.append(set_data)
    driver.quit()
    time.sleep(0.05)

with open(f'./shoplook/data/casual_data.json', 'w') as fout:
    json.dump(data, fout)

  0%|          | 0/250 [00:00<?, ?it/s]

SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/75e1d887-e308-4484-b0b7-68cdc2e71a1a.jpeg - as ./shoplook/data/images/casual/4d2ed9fcca.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/0a6eefbc-512a-429e-8d20-2d987964b3db.jpeg - as ./shoplook/data/images/casual/06c1bb1183.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/5edda2a8-b3b9-425a-b338-6bb1203c53f3.jpeg - as ./shoplook/data/images/casual/0d3173d352.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/2839c608-2a3a-47b7-a069-a3e951a3b6ba.jpeg - as ./shoplook/data/images/casual/a9c6597490.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/79d2cef6-0357-4e15-8091-2cfd28eee98f.jpeg - as ./shoplook/data/images/casual/af6c77f72b.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/a14f24e3-cdeb-49a4

  0%|          | 1/250 [00:15<1:02:31, 15.07s/it]

SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/01608eea-0131-4584-b9a9-5c80fefd28be.jpeg - as ./shoplook/data/images/casual/ce69c11132.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/ad24ba67-e6cf-4911-b84c-0ba0a90858cc.jpeg - as ./shoplook/data/images/casual/5fe2ae83c0.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/a9058fab-661d-494e-80b2-481d96776de9.jpeg - as ./shoplook/data/images/casual/8fb869fb6e.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/bcade774-aa95-45ad-9dec-ac3e910c2344.webp - as ./shoplook/data/images/casual/a1e01cc180.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/84f60239-e571-4a31-939f-fe6efee30f02.jpeg - as ./shoplook/data/images/casual/24d3aa5dbe.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/cb08a6e4-8581-4c4e

  1%|          | 2/250 [00:30<1:02:20, 15.08s/it]

...

SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/0195c639-a248-4513-a46b-f5741a03e958.jpeg - as ./shoplook/data/images/casual/11cc1c7c84.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/4de3df4c-99dd-4f60-af46-b21c0ffbabc3.jpg - as ./shoplook/data/images/casual/c2df11e095.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/621b2fd3-9345-44ef-96f6-763eb36aa925.jpg - as ./shoplook/data/images/casual/43c7417c5c.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/83cf0c2a-e670-46f6-9c17-a3a3f4eccec4.jpeg - as ./shoplook/data/images/casual/baee2e4fe6.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/5bd255a9-0bc5-4c8e-a0d2-6569d4091335.jpeg - as ./shoplook/data/images/casual/64877efb62.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/101ac564-44d4-4856-b

 25%|██▍       | 62/250 [15:27<46:57, 14.99s/it]

SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/667b8b42-005e-4009-89f5-84a33ca6932a.jpeg - as ./shoplook/data/images/casual/ad50b86043.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/51e687f7-4956-4fc1-b0e5-f5c5d6ebfb90.jpeg - as ./shoplook/data/images/casual/34aa52cd85.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/4402cc19-d718-4d83-aa24-1d7ae79f9b0c.jpeg - as ./shoplook/data/images/casual/2bf7f2946a.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/8178cb2c-0daa-4b64-886e-c369beef715c.jpeg - as ./shoplook/data/images/casual/0199e4f4b8.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/1efbbb52-d17c-4904-8285-c35e1c0ebb1f.jpeg - as ./shoplook/data/images/casual/a87f599a9f.jpg
SUCCESS - saved https://d2bzx2vuetkzse.cloudfront.net/fit-in/0x450/unshoppable_producs/55474ac2-9fd0-4409

 25%|██▌       | 63/250 [16:01<47:32, 15.25s/it]


KeyboardInterrupt: 

In [30]:
with open(f'./shoplook/data/casual_data.json', 'w') as fout:
    json.dump(data, fout)